In [4]:
#Importación de librerías y extracción de data

import requests
from bs4 import BeautifulSoup
import os
import re

# URL de la página
url = "https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

# Realizar la solicitud GET a la página
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Crear una carpeta para almacenar los archivos descargados
os.makedirs('parquet_files', exist_ok=True)

# Función para verificar si el archivo está dentro del rango de años
def is_within_year_range(file_name, start_year, end_year):
    match = re.search(r'(\d{4})', file_name)
    if match:
        year = int(match.group(1))
        return start_year >= year >= end_year
    return False

# Rango de años
start_year = 2023
end_year = 2017

# Buscar todos los enlaces de archivos Parquet
for link in soup.find_all('a', href=True):
    href = link['href']
    if href.endswith('.parquet'):
        file_name = href.split('/')[-1]
        # Filtrar solo archivos que contengan 'green_tripdata' o 'yellow_tripdata' y estén en el rango de años
        if ('green_tripdata' in file_name or 'yellow_tripdata' in file_name) and is_within_year_range(file_name, start_year, end_year):
            # Descargar el archivo
            file_url = href
            file_response = requests.get(file_url)
            with open(os.path.join('parquet_files', file_name), 'wb') as file:
                file.write(file_response.content)
            
            print(f"Descargado: {file_name}")

print("Descarga completada.")


Descargado: yellow_tripdata_2023-01.parquet
Descargado: green_tripdata_2023-01.parquet
Descargado: yellow_tripdata_2023-02.parquet
Descargado: green_tripdata_2023-02.parquet
Descargado: yellow_tripdata_2023-03.parquet
Descargado: yellow_tripdata_2023-04.parquet
Descargado: green_tripdata_2023-04.parquet
Descargado: yellow_tripdata_2023-06.parquet
Descargado: green_tripdata_2023-06.parquet
Descargado: yellow_tripdata_2023-12.parquet
Descargado: green_tripdata_2023-12.parquet
Descargado: yellow_tripdata_2022-01.parquet
Descargado: green_tripdata_2022-01.parquet
Descargado: yellow_tripdata_2022-02.parquet
Descargado: green_tripdata_2022-02.parquet
Descargado: yellow_tripdata_2022-03.parquet
Descargado: green_tripdata_2022-03.parquet
Descargado: yellow_tripdata_2022-04.parquet
Descargado: green_tripdata_2022-04.parquet
Descargado: yellow_tripdata_2022-05.parquet
Descargado: green_tripdata_2022-05.parquet
Descargado: yellow_tripdata_2022-06.parquet
Descargado: green_tripdata_2022-06.parquet

In [1]:
#Leer archivo parquet

#importacion de librerias
import pandas as pd

#rutas a los archivos
file_path_greentaxis = './parquet_files/green_tripdata_2023-06.parquet'
file_path_yellowtaxis = './parquet_files/yellow_tripdata_2023-06.parquet'

#dataframe
df_green_taxis = pd.read_parquet(file_path_greentaxis)
df_yellow_taxis = pd.read_parquet(file_path_yellowtaxis)

In [4]:
df_green_taxis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65550 entries, 0 to 65549
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   VendorID               65550 non-null  int32         
 1   lpep_pickup_datetime   65550 non-null  datetime64[us]
 2   lpep_dropoff_datetime  65550 non-null  datetime64[us]
 3   store_and_fwd_flag     60359 non-null  object        
 4   RatecodeID             60359 non-null  float64       
 5   PULocationID           65550 non-null  int32         
 6   DOLocationID           65550 non-null  int32         
 7   passenger_count        60359 non-null  float64       
 8   trip_distance          65550 non-null  float64       
 9   fare_amount            65550 non-null  float64       
 10  extra                  65550 non-null  float64       
 11  mta_tax                65550 non-null  float64       
 12  tip_amount             65550 non-null  float64       
 13  t

In [5]:
df_yellow_taxis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3307234 entries, 0 to 3307233
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int32         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int32         
 8   DOLocationID           int32         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  Airport_fee           

In [2]:
# Seleccionar las columnas necesarias y agregar la columna 'type_taxi'
df_green_taxis['type_taxi'] = 'green'
df_green_taxis = df_green_taxis[['lpep_pickup_datetime', 'lpep_dropoff_datetime', 'PULocationID', 'DOLocationID', 'trip_distance', 'fare_amount', 'type_taxi']]
df_yellow_taxis['type_taxi'] = 'yellow'
df_yellow_taxis = df_yellow_taxis[['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'PULocationID', 'DOLocationID', 'trip_distance', 'fare_amount', 'type_taxi']]

In [3]:
# Renombrar las columnas de tiempo para que coincidan
df_green_taxis = df_green_taxis.rename(columns={'lpep_pickup_datetime': 'pickup_time', 'lpep_dropoff_datetime': 'dropoff_time'})
df_yellow_taxis = df_yellow_taxis.rename(columns={'tpep_pickup_datetime': 'pickup_time', 'tpep_dropoff_datetime': 'dropoff_time'})


In [4]:
# Extraer la fecha y la hora de las columnas pickup_time y dropoff_time
df_green_taxis['date'] = df_green_taxis['pickup_time'].dt.date
df_green_taxis['pickup_time'] = df_green_taxis['pickup_time'].dt.time
df_green_taxis['dropoff_time'] = df_green_taxis['dropoff_time'].dt.time

df_yellow_taxis['date'] = df_yellow_taxis['pickup_time'].dt.date
df_yellow_taxis['pickup_time'] = df_yellow_taxis['pickup_time'].dt.time
df_yellow_taxis['dropoff_time'] = df_yellow_taxis['dropoff_time'].dt.time

In [5]:
# Concatenar los dos datasets
trip_data = pd.concat([df_green_taxis, df_yellow_taxis], ignore_index=True)

In [6]:
# Reordenar las columnas para que 'date' esté al principio
trip_data = trip_data[['date', 'pickup_time', 'dropoff_time', 'PULocationID', 'DOLocationID', 'trip_distance', 'fare_amount', 'type_taxi']]

In [7]:
# Ordenar el DataFrame por pickup_time
trip_data = trip_data.sort_values(by=['date', 'pickup_time']).reset_index(drop=True)

In [10]:
# Función para extraer mes y año de la ruta del archivo
import re
def obtener_mes_y_ano(file_path):
    match = re.search(r'(\d{4})-(\d{2})', file_path)
    if match:
        año_referencia = int(match.group(1))
        mes_referencia = int(match.group(2))
        return mes_referencia, año_referencia
    else:
        raise ValueError("No se pudo extraer el mes y año del nombre del archivo.")

# Obtener mes y año de referencia para cada archivo
mes_referencia_green, año_referencia_green = obtener_mes_y_ano(file_path_greentaxis)
mes_referencia_yellow, año_referencia_yellow = obtener_mes_y_ano(file_path_yellowtaxis)

# Asumiendo que ambos archivos tienen el mismo mes y año, puedes usar uno de ellos
mes_referencia = mes_referencia_green
año_referencia = año_referencia_green

print(f'Mes de referencia: {mes_referencia}')
print(f'Año de referencia: {año_referencia}')

Mes de referencia: 6
Año de referencia: 2023


In [11]:
# Si 'date' es datetime
trip_data['date'] = pd.to_datetime(trip_data['date'])

# Filtrar por mes y año
trip_data_filtrado = trip_data[(trip_data['date'].dt.year == año_referencia) & (trip_data['date'].dt.month == mes_referencia)]

In [13]:
# Resetear el índice después de filtrar
trip_data_filtrado = trip_data_filtrado.reset_index(drop=True)

In [14]:
# Mostrar el dataframe resultante
print(trip_data_filtrado.head())

        date pickup_time dropoff_time  PULocationID  DOLocationID  \
0 2023-06-01    00:00:00     00:07:33           186           230   
1 2023-06-01    00:00:00     16:48:00           158           264   
2 2023-06-01    00:00:00     17:24:53           141           264   
3 2023-06-01    00:00:03     00:09:17           237           263   
4 2023-06-01    00:00:03     00:20:33           249           140   

   trip_distance  fare_amount type_taxi  
0           1.10          8.6    yellow  
1          29.74         70.0    yellow  
2           2.22         14.9    yellow  
3           1.62         10.7    yellow  
4           4.67         22.6    yellow  
